In [0]:
import numpy as np

In [0]:
class Ban:
    def init_ban(self):
        first_board = np.array([[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1, 0, 0, 0, 0, 0, 0, 0, 0,-1]
                              ,[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]])
        first_board[4][4] = first_board[5][5] = 2
        first_board[4][5] = first_board[5][4] = 1
        return first_board

    def stone(self,ishi):
        if ishi == 0:
            return '・'
        elif ishi == 1:
            return '●'
        else:
            return'○'
    def draw_ban(self,board):
        print('  a  b  c  d  e  f  g  h ')
        print('  ------------------------')
        for i in range(1,9):
            print('%d|'%i, end='')
            for j in range(1,9):
                print(self.stone(board[i][j]),end=' ')
            print('|')
        print('  ------------------------')


In [0]:
class Play:
    def count_turn_over(self, board, color, y, x, d, e):
        i = 1
        while board[y + i * d][x + i * e] == (3 - color):
            i += 1
        if board[y + i * d][x + i * e] == color:
            return i - 1
        else:
            return 0
    def is_legal(self,board, color, y, x):
        if x < 1 or y > 8 or x < 1 or y > 8: return 0
        if board[y][x] != 0 :return 0
        if self.count_turn_over(board, color, y, x, -1, 0): return 1
        if self.count_turn_over(board, color, y, x,  1, 0): return 1
        if self.count_turn_over(board, color, y, x,  0,-1): return 1
        if self.count_turn_over(board, color, y, x,  0, 1): return 1
        if self.count_turn_over(board, color, y, x, -1,-1): return 1
        if self.count_turn_over(board, color, y, x, -1, 1): return 1
        if self.count_turn_over(board, color, y, x,  1,-1): return 1
        if self.count_turn_over(board, color, y, x,  1, 1): return 1
        return 0
    
    def legal_list(self,board,color):
        legal = np.empty((0,2), int)
        for i in range(1, 9):
          for j in range(1, 9):
            if self.is_legal(board, color, i, j):
                legal = np.append(legal, np.array([[i,j]]), axis = 0)

        return legal
              
    def exist_legal_move(self, board, color):
        for i in range(1,9):
            for j in range(1,9):
                if self.is_legal(board, color, i, j):return 1
        return 0
    
    def set_turn(self, board, color, y, x):
        count = 0
        for d in range(-1,2):
            for e in range(-1,2):
                if (d == 0 and e == 0):
                    continue
                count = self.count_turn_over(board, color, y, x, d, e)
                for i in range(1, count+1):
                    board[y + i * d][x + i * e] = color
        board[y][x] = color

In [0]:
class Cpu:
    def cpu_randmove(self, board, color):
        l = Play().legal_list(board, color)
        if l.shape[0] == 1:
          y, x = l[0]
        else:
          y, x = l[int(np.random.randint(l.shape[0] - 1,size = 1))]
        return y, x

In [0]:
class Othello:
    def game(self):
        ban = Ban().init_ban()
        color = 1
        print('ゲームスタート！')
        while True:
            Ban().draw_ban(ban)
            if not Play().exist_legal_move(ban, color):
                print('打つ手が無いのでパスします')
                color = (3-color)
                if not Play().exist_legal_move(ban, color):
                    print('打つ手が無いのでパスします')
                    break
            y, x = self.get_move(ban, color)
            Play().set_turn(ban, color, y, x)
            
            color = int(3 - color)
        c_1 = np.sum(ban == 1)
        c_2 = np.sum(ban == 2)
        print('ゲーム終了')
        print('Player 1 %d' % c_1)
        print('Player 2 %d' % c_2)
        if c_1 > c_2:
          print('勝者:Player 1')
        elif c_1<c_2:
          print('勝者:Player 2')
        elif c_1 == c_2:
          print('引き分け')
          
    def cvc_game(self):
        ban = Ban().init_ban()
        color = 1
        while True:
          Ban().draw_ban(ban)
          if not Play().exist_legal_move(ban, color):
            print('打つ手が無いのでパスします')
            color = (3 - color)
            if not Play().exist_legal_move(ban, color):
                print('打つ手が無いのでパスします')
                break
          y, x = Cpu().cpu_randmove(ban, color)
          Play().set_turn(ban, color, y, x)
          color = int(3 - color)
        c_1 = np.sum(ban == 1)
        c_2 = np.sum(ban == 2)
        print('ゲーム終了')
        print('Player 1 %d' % c_1)
        print('Player 2 %d' % c_2)
        if c_1 > c_2:
          print('勝者:Player 1')
        elif c_1 < c_2:
          print('勝者:Player 2')
        elif c_1 == c_2:
          print('引き分け')
          
    def get_move(self, board, color):
        d = {'a':1, 'b':2, 'c':3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8}
        p = {1 : '●', 2 : '○'}
        if color == 1:
            print('Player {0[1]}(半角数字[半角スペース]半角英字)'.format(p), end = '')
        elif color == 2:
            print('Player {0[2]}(半角数字[半角スペース]半角英字)'.format(p), end='')
        while True:
            print('>>',end = '')
            Y,X = input().split()
            y = int(Y)
            x = d[X]
            if (Play().is_legal(board, color, y, x)):
                return y,x
            else:
                print('illegal hand!', end = '')

In [98]:
if __name__ == '__main__':
  #Othello().game()
  Othello().cvc_game()
  

  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ・ ・ ・ ・ ・ ・ ・ |
4|・ ・ ・ ○ ● ・ ・ ・ |
5|・ ・ ・ ● ○ ・ ・ ・ |
6|・ ・ ・ ・ ・ ・ ・ ・ |
7|・ ・ ・ ・ ・ ・ ・ ・ |
8|・ ・ ・ ・ ・ ・ ・ ・ |
  ------------------------
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ・ ・ ● ・ ・ ・ ・ |
4|・ ・ ・ ● ● ・ ・ ・ |
5|・ ・ ・ ● ○ ・ ・ ・ |
6|・ ・ ・ ・ ・ ・ ・ ・ |
7|・ ・ ・ ・ ・ ・ ・ ・ |
8|・ ・ ・ ・ ・ ・ ・ ・ |
  ------------------------
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ・ ○ ● ・ ・ ・ ・ |
4|・ ・ ・ ○ ● ・ ・ ・ |
5|・ ・ ・ ● ○ ・ ・ ・ |
6|・ ・ ・ ・ ・ ・ ・ ・ |
7|・ ・ ・ ・ ・ ・ ・ ・ |
8|・ ・ ・ ・ ・ ・ ・ ・ |
  ------------------------
  a  b  c  d  e  f  g  h 
  ------------------------
1|・ ・ ・ ・ ・ ・ ・ ・ |
2|・ ・ ・ ・ ・ ・ ・ ・ |
3|・ ● ● ● ・ ・ ・ ・ |
4|・ ・ ・ ○ ● ・ ・ ・ |
5|・ ・ ・ ● ○ ・ ・ ・ |
6|・ ・ ・ ・ ・ ・ ・ ・ |
7|・ ・ ・ ・ ・ ・ ・ ・ |
8|・ ・ ・ ・ ・ ・ ・ ・ |
  ------------------------
  a  b  c  d  e  f  g  h 
  ------------